In [1]:
import pandas as pd
import requests

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
admis.iloc[:,6:8]

,Attending Provider NPI,Facility Provider NPI
0,1417170523,1306890389
1,1669599197,1902803315
2,1407880511,1902803315
3,1407008907,1427055839
4,1518045608,1992818256
...,...,...
6208,1720043417,1689772592
6209,1215968474,1427055839
6210,1861831836,1689608150
6211,1750544516,1689608150


In [4]:
visits.iloc[:,8:12]

,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI
0,NaN,NaN,1487856779,1043427248
1,1528030996,1902803315,1528030996,1043427248
2,NaN,NaN,1477579365,1043427248
3,1821178088,1104842020,1821178088,1407058787
4,NaN,NaN,1609004191,1184722779
...,...,...,...,...
231368,NaN,NaN,1851738108,1184722779
231369,NaN,NaN,1215163035,1043427248
231370,NaN,NaN,1215163035,1043427248
231371,NaN,NaN,1144252396,1962790014


In [25]:
npi = admis.iloc[:,6].append((admis.iloc[:,7], visits.iloc[:,8],
                              visits.iloc[:,9],visits.iloc[:,10],
                              visits.iloc[:,11]), ignore_index=True)

In [26]:
npi = npi.drop_duplicates().reset_index().dropna().rename(columns={0:'NPI'}).drop(columns=['index'])
npi

,NPI
0,1417170523
1,1669599197
2,1407880511
3,1407008907
4,1518045608
...,...
5411,1336222397
5412,1336150572
5413,1417017435
5414,1790964864


In [27]:
npi.iloc[2,0]

'1407880511'

In [28]:
endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
params = {'number' : '1306890389'}

In [29]:
response = requests.get(endpoint , params = params)

In [30]:
response

<Response [200]>

In [31]:
res = response.json()
res

{'result_count': 1,
 'results': [{'enumeration_type': 'NPI-2',
   'number': 1306890389,
   'last_updated_epoch': 1620655474,
   'created_epoch': 1148256000,
   'basic': {'organization_name': 'LOS ROBLES REGIONAL MEDICAL CENTER',
    'organizational_subpart': 'NO',
    'enumeration_date': '2006-05-22',
    'last_updated': '2021-05-10',
    'status': 'A',
    'authorized_official_first_name': 'DEBBIE',
    'authorized_official_last_name': 'HERWALDT',
    'authorized_official_telephone_number': '805-370-4421',
    'authorized_official_title_or_position': 'CFO',
    'name': 'LOS ROBLES REGIONAL MEDICAL CENTER',
    'certification_date': '2021-05-10'},
   'other_names': [{'organization_name': 'LOS ROBLES HOSPITAL & MEDICAL CENTER',
     'code': '3',
     'type': 'Doing Business As'}],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '215 W JANSS RD',
     'address_2': '',
     'cit

In [32]:
res.keys()

dict_keys(['result_count', 'results'])

In [33]:
res['results'][0]

{'enumeration_type': 'NPI-2',
 'number': 1306890389,
 'last_updated_epoch': 1620655474,
 'created_epoch': 1148256000,
 'basic': {'organization_name': 'LOS ROBLES REGIONAL MEDICAL CENTER',
  'organizational_subpart': 'NO',
  'enumeration_date': '2006-05-22',
  'last_updated': '2021-05-10',
  'status': 'A',
  'authorized_official_first_name': 'DEBBIE',
  'authorized_official_last_name': 'HERWALDT',
  'authorized_official_telephone_number': '805-370-4421',
  'authorized_official_title_or_position': 'CFO',
  'name': 'LOS ROBLES REGIONAL MEDICAL CENTER',
  'certification_date': '2021-05-10'},
 'other_names': [{'organization_name': 'LOS ROBLES HOSPITAL & MEDICAL CENTER',
   'code': '3',
   'type': 'Doing Business As'}],
 'addresses': [{'country_code': 'US',
   'country_name': 'United States',
   'address_purpose': 'LOCATION',
   'address_type': 'DOM',
   'address_1': '215 W JANSS RD',
   'address_2': '',
   'city': 'THOUSAND OAKS',
   'state': 'CA',
   'postal_code': '913601847',
   'telepho

In [34]:
res['results'][0].keys()

dict_keys(['enumeration_type', 'number', 'last_updated_epoch', 'created_epoch', 'basic', 'other_names', 'addresses', 'taxonomies', 'identifiers', 'endpoints'])

In [35]:
res['results'][0]['enumeration_type']

'NPI-2'

In [42]:
endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
enumeration = list()
entry = []
for x in range(0,100):
    params = {'number' : npi.iloc[x,0]}
    print(str(x) + str(params))
    response = requests.get(endpoint , params = params)
    res = response.json()
    enumeration.append(res['results'][0]['enumeration_type'])
    entry.append(res['results'][0])

0{'number': '1417170523'}
1{'number': '1669599197'}
2{'number': '1407880511'}
3{'number': '1407008907'}
4{'number': '1518045608'}
5{'number': '1346556891'}
6{'number': '1316989247'}
7{'number': '1417995317'}
8{'number': '1396818134'}
9{'number': '1447495387'}
10{'number': '1528163003'}
11{'number': '1366604969'}
12{'number': '1427375757'}
13{'number': '1619073954'}
14{'number': '1972526721'}
15{'number': '1366847147'}
16{'number': '1750346474'}
17{'number': '1932127248'}
18{'number': '1346566221'}
19{'number': '1598993990'}
20{'number': '1720240526'}
21{'number': '1023149044'}
22{'number': '1508841602'}
23{'number': '1730127069'}
24{'number': '1053334292'}
25{'number': '1922323856'}
26{'number': '1700828704'}
27{'number': '1982865044'}
28{'number': '1902850498'}
29{'number': '1710270442'}
30{'number': '1427154533'}
31{'number': '1306902739'}
32{'number': '1518963032'}
33{'number': '1750425278'}
34{'number': '1427344951'}
35{'number': '1073566931'}
36{'number': '1679541775'}
37{'number'

In [40]:
enumeration

['NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1',
 'NPI-1']

In [41]:
entry

[{'enumeration_type': 'NPI-1',
  'number': 1417170523,
  'last_updated_epoch': 1394755200,
  'created_epoch': 1176163200,
  'basic': {'name_prefix': 'DR.',
   'first_name': 'LEENA',
   'last_name': 'NATHAN',
   'middle_name': 'SHANKAR',
   'credential': 'M.D.',
   'sole_proprietor': 'YES',
   'gender': 'F',
   'enumeration_date': '2007-04-10',
   'last_updated': '2014-03-14',
   'status': 'A',
   'name': 'NATHAN LEENA'},
  'other_names': [],
  'addresses': [{'country_code': 'US',
    'country_name': 'United States',
    'address_purpose': 'LOCATION',
    'address_type': 'DOM',
    'address_1': '1250 LA VENTA DR',
    'address_2': 'SUITE 105',
    'city': 'WESTLAKE VILLAGE',
    'state': 'CA',
    'postal_code': '913613702',
    'telephone_number': '805-557-7180',
    'fax_number': '805-557-7181'},
   {'country_code': 'US',
    'country_name': 'United States',
    'address_purpose': 'MAILING',
    'address_type': 'DOM',
    'address_1': '1250 LA VENTA DR',
    'address_2': 'SUITE 105',
